In [4]:
import pandas as pd
pd.set_option('precision', 3)
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import os
import numpy as np

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

import mxnet as mx
from gluonts.model.n_beats import NBEATSEnsembleEstimator
from gluonts.mx import Trainer
from gluonts.evaluation import make_evaluation_predictions

In [5]:
"""
Set sample rate. In this notebook, all data will be resampled at the chosen frequency.
'MS' : Monthly (Month Start)
'W' : Weekly
'D' : Daily
"""

year_period = {'MS': 12, 'W': 52, 'D': 365}
frequency = 'MS'
one_year = year_period[frequency]
output_path = "./output/nbeatsfredvars_TEST"
if not os.path.exists(output_path):
    os.mkdir(output_path)

## Load Data From File

In [6]:
foodprice_categories = pd.read_csv("./foodprice_categories.txt", sep='\n', header=None)[0].to_list()
foodprice_df = pd.read_csv("./all_data.csv", index_col=0)
foodprice_df = foodprice_df.set_index(pd.DatetimeIndex(foodprice_df.index))
foodprice_df

,Bakery and cereal products (excluding baby food),Dairy products and eggs,"Fish, seafood and other marine products",Food purchased from restaurants,Food,"Fruit, fruit preparations and nuts",Meat,Other food products and non-alcoholic beverages,Vegetables and vegetable preparations,DEXCAUS,...,XTIMVA01CAM657S,XTIMVA01CAM659S,XTIMVA01CAM664N,XTIMVA01CAM664S,XTIMVA01CAM667S,XTNTVA01CAM664N,XTNTVA01CAM664S,XTNTVA01CAM667S,TOTALNS,TOTALSL
1986-01-01,69.3,70.9,60.6,59.1,67.3,76.0,65.1,77.5,76.0,1.392,...,3.644,12.051,9.368e+09,9.496e+09,6.749e+09,6.906e+08,1.052e+09,7.479e+08,607.369,605.703
1986-02-01,70.3,70.8,61.3,59.1,66.9,77.6,64.2,78.1,68.4,1.392,...,1.965,16.745,9.495e+09,9.632e+09,6.881e+09,-9.880e+07,1.539e+08,1.099e+08,605.807,610.678
1986-03-01,70.6,71.1,61.3,59.3,67.0,79.2,64.2,78.6,66.2,1.392,...,-11.565,1.655,8.803e+09,8.529e+09,6.085e+09,9.138e+08,9.079e+08,6.478e+08,606.799,613.377
1986-04-01,71.3,71.0,61.4,59.7,67.7,82.2,63.6,79.5,71.1,1.392,...,13.334,10.821,1.034e+10,9.569e+09,6.897e+09,3.470e+08,6.563e+08,4.730e+08,614.367,619.658
1986-05-01,71.2,71.4,61.9,59.9,68.2,83.5,64.0,79.8,75.3,1.377,...,-4.236,6.160,9.598e+09,9.091e+09,6.605e+09,7.013e+08,6.893e+08,5.008e+08,621.915,625.820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,157.7,145.3,146.2,163.9,156.8,144.5,176.7,142.2,153.4,1.203,...,-1.087,31.561,5.159e+10,4.979e+10,4.075e+10,3.661e+09,3.111e+09,2.546e+09,4259.604,4307.137
2021-07-01,157.9,146.4,146.6,165.2,157.6,141.7,180.9,141.9,154.8,1.240,...,1.990,18.580,5.106e+10,5.205e+10,4.156e+10,3.780e+08,1.129e+09,9.018e+08,4276.202,4323.730
2021-08-01,158.5,148.3,146.8,165.9,158.0,142.5,182.1,141.7,152.2,1.251,...,-2.312,14.723,5.245e+10,5.115e+10,4.060e+10,1.300e+09,2.674e+09,2.123e+09,4316.303,4337.489
2021-09-01,158.1,148.0,147.1,165.9,158.5,141.5,184.8,144.3,150.0,1.262,...,-3.461,8.557,5.080e+10,4.965e+10,3.920e+10,1.561e+09,2.277e+09,1.798e+09,4346.662,4367.402


# NBEATS Model and Experiments

## Data Splitting

For each such candidate forecast, we should record any uncertainty/confidence metrics it provides, and evaluation metrics for that same model configuration over the test set. i.e. When model configuration XYZ was used to forecast Meat prices over the test set (with that data not being used for training or validation!) - what were its evaluation metrics on the withheld data? We should report this consistently for ALL EXPERIMENTS. 

For all models, we will use the following "simulated" report dates. This is a form of cross validation over time. We train a model up to each cutoff date, and then produce and evaluate 18-month forecasts. We can then collect each model's validation metric, take the mean, and use this to do model selection for the final forecast (or ensemble of forecasts!).

In [17]:
report_sim_dates = pd.read_csv("./reportsimdates.txt", header=None)[0].to_list()
report_sim_dates

['2015-07-01',
 '2016-07-01',
 '2017-07-01',
 '2018-07-01',
 '2019-07-01',
 '2020-07-01']

In [32]:
sim_train_dates = {}
sim_valid_dates = {}

for date in report_sim_dates:
    sim_train_dates[date] = foodprice_df.index[foodprice_df.index <= date]
    sim_valid_dates[date] = foodprice_df.index[(foodprice_df.index > date) & (foodprice_df.index <= (pd.to_datetime(date) + pd.DateOffset(months=18)))]

## Fitting and Evaluating a Single NBEATS Model: Example Using All Food Prices

In [33]:
N = foodprice_df.shape[1]
T = foodprice_df.shape[0]
prediction_length = 18
freq = "MS"
dataset = foodprice_df.T.values
start = pd.Timestamp("2016-07-01", freq=freq)

/tmp/ipykernel_5774/56946274.py:6: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  start = pd.Timestamp("2016-07-01", freq=freq)


In [34]:
from gluonts.dataset.common import ListDataset

In [35]:
dataset_df = foodprice_df.T
dataset_df

,1986-04-01,1986-05-01,1986-06-01,1986-07-01,1986-08-01,1986-09-01,1986-10-01,1986-11-01,1986-12-01,1987-01-01,...,2020-12-01,2021-01-01,2021-02-01,2021-03-01,2021-04-01,2021-05-01,2021-06-01,2021-07-01,2021-08-01,2021-09-01
Bakery and cereal products (excluding baby food),7.130e+01,7.120e+01,7.110e+01,7.170e+01,7.190e+01,7.170e+01,7.110e+01,7.160e+01,7.220e+01,7.260e+01,...,1.564e+02,1.542e+02,1.571e+02,1.568e+02,1.562e+02,1.578e+02,1.577e+02,1.579e+02,1.585e+02,1.581e+02
Dairy products and eggs,7.100e+01,7.140e+01,7.110e+01,7.130e+01,7.150e+01,7.180e+01,7.180e+01,7.210e+01,7.250e+01,7.250e+01,...,1.415e+02,1.416e+02,1.431e+02,1.449e+02,1.461e+02,1.466e+02,1.453e+02,1.464e+02,1.483e+02,1.480e+02
"Fish, seafood and other marine products",6.140e+01,6.190e+01,6.200e+01,6.220e+01,6.270e+01,6.310e+01,6.360e+01,6.530e+01,6.590e+01,6.710e+01,...,1.447e+02,1.434e+02,1.439e+02,1.449e+02,1.451e+02,1.476e+02,1.462e+02,1.466e+02,1.468e+02,1.471e+02
Food purchased from restaurants,5.970e+01,5.990e+01,6.000e+01,6.060e+01,6.090e+01,6.090e+01,6.130e+01,6.160e+01,6.170e+01,6.210e+01,...,1.616e+02,1.626e+02,1.629e+02,1.626e+02,1.632e+02,1.635e+02,1.639e+02,1.652e+02,1.659e+02,1.659e+02
Food,6.770e+01,6.820e+01,6.840e+01,6.920e+01,6.950e+01,6.990e+01,7.020e+01,7.060e+01,7.050e+01,7.120e+01,...,1.536e+02,1.550e+02,1.556e+02,1.555e+02,1.554e+02,1.566e+02,1.568e+02,1.576e+02,1.580e+02,1.585e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XTNTVA01CAM664N,3.470e+08,7.013e+08,5.973e+08,-4.890e+08,9.390e+07,1.542e+09,1.262e+09,1.192e+09,1.406e+09,3.872e+08,...,8.227e+08,3.783e+09,2.130e+09,-7.588e+08,-8.233e+08,-3.382e+08,3.639e+09,2.321e+08,1.505e+09,1.505e+09
XTNTVA01CAM664S,6.563e+08,6.893e+08,5.314e+08,-9.670e+07,2.959e+08,1.222e+09,1.165e+09,6.931e+08,8.879e+08,6.596e+08,...,-1.381e+09,2.116e+09,1.767e+09,-9.810e+07,1.340e+09,-4.385e+08,3.230e+09,1.134e+09,2.960e+09,2.960e+09
XTNTVA01CAM667S,4.730e+08,5.008e+08,3.823e+08,-7.004e+07,2.131e+08,8.806e+08,8.390e+08,4.999e+08,6.435e+08,4.850e+08,...,-1.078e+09,1.663e+09,1.392e+09,-7.802e+07,1.072e+09,-3.617e+08,2.643e+09,9.054e+08,2.350e+09,2.350e+09
TOTALNS,6.144e+02,6.219e+02,6.279e+02,6.336e+02,6.405e+02,6.499e+02,6.567e+02,6.569e+02,6.664e+02,6.553e+02,...,4.185e+03,4.177e+03,4.166e+03,4.167e+03,4.188e+03,4.233e+03,4.268e+03,4.285e+03,4.326e+03,4.326e+03


In [36]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

metrics = {
    'r2_score': r2_score,
    'mae': mean_absolute_error,
    'mape': mean_absolute_percentage_error,
    'mse': mean_squared_error,
    'rmse': rmse
}

In [37]:
def train_eval_nbeats(report_sim_date):

    report_train_dates = sim_train_dates[report_sim_date]
    report_valid_dates = sim_valid_dates[report_sim_date]
    
    # train dataset: cut the last window of length "prediction_length", add "target" and "start" fields
    train_ds = ListDataset(
        [{'target': x, 'start': start} for x in dataset_df[list(report_train_dates)].values],
        freq=freq
    )

    valid_ds_report = ListDataset(
        [{'target': x, 'start': start} for x in dataset_df[list(report_train_dates) + list(report_valid_dates)].values],
        freq='MS'
    )

    estimator = NBEATSEnsembleEstimator(
        prediction_length=prediction_length,
        meta_bagging_size = 3,  
        meta_context_length = [prediction_length * m for m in [3,5,7]],
        meta_loss_function = ['sMAPE'], 
        num_stacks = 30,
        widths= [512],
        freq="MS",
        trainer=Trainer(
                    epochs=100,
                    num_batches_per_epoch=200,
                    batch_size=16,
                    ctx=mx.context.gpu()
                )
    )

    predictor = estimator.train(train_ds)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=valid_ds_report,  # test dataset
        predictor=predictor,  # predictor
    )

    forecasts = list(forecast_it)
    all_fc_dates = list(report_train_dates) + list(report_valid_dates)

    all_food_metrics = {}
    food_forecasts = {}

    for target_index in range(len(forecasts)):

        # Get food price category
        foodprice_category = foodprice_df.columns[target_index]

        if foodprice_category in foodprice_categories:

            # plot actual
            fig, ax = plt.subplots(figsize=(8,3))
            ax.plot(all_fc_dates, foodprice_df[foodprice_category][all_fc_dates])

            # plot forecast
            forecast_entry = forecasts[target_index]
            ax.plot(report_valid_dates, forecast_entry.mean[:len(report_valid_dates)], color='green')

            plt.title(f"{foodprice_category}, {report_sim_date}")
            plt.grid()
            plt.show()

            fc_metrics = pd.Series({metric_name: metric_fn(y_true=foodprice_df[foodprice_category][report_valid_dates], y_pred=forecast_entry.mean[:len(report_valid_dates)]) for metric_name, metric_fn in metrics.items()})
            print(fc_metrics)

            all_food_metrics[foodprice_category] = fc_metrics
            food_forecasts[foodprice_category] = pd.Series(forecast_entry.mean[:len(report_valid_dates)], index=report_valid_dates, name=foodprice_category)

    all_forecasts = pd.DataFrame(food_forecasts)
    all_forecasts.to_csv(f"{output_path}/forecasts_{report_sim_date}.csv")

    return all_food_metrics, all_forecasts

In [38]:
all_valid_metrics = {}
all_forecasts = {}

for report_sim_date in report_sim_dates:
    valid_metrics, forecasts = train_eval_nbeats(report_sim_date)
    all_valid_metrics[report_sim_date] = valid_metrics
    all_forecasts[report_sim_date] = forecasts

In [39]:
valid_metrics_concat = {}

all_valid_metrics.keys()

for report_date, valid_scores in all_valid_metrics.items():
    valid_metrics_concat[report_date] = pd.DataFrame(valid_scores).T
index = valid_metrics_concat[report_date].index
columns = valid_metrics_concat[report_date].columns
scores = [df.values for date, df in valid_metrics_concat.items()]
mean_scores = pd.DataFrame(np.array(scores).mean(axis=0), index=index, columns=columns)
mean_scores.to_csv(f"{output_path}/mean_fc_valid_metrics.csv")
mean_scores